In [121]:
import datetime
import numpy as np
import pandas as pd
import dateutil
import copy
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class RateConverter:
    def mu_month2year(r):
        return np.power(r, 12)
    # end def
    
    def mu_year2month(R):
        return np.exp(np.log(R) / 12)
    # end def
    
    def std_month2year(s):
        return s*np.sqrt(12)
    # end def
    
    def std_year2month(S):
        return S/np.sqrt(12)
    # end def    
# end class

In [4]:
def startOfMonth(dt):
    return datetime.datetime(dt.year, dt.month, 1)
# end def     

In [5]:
class GeneralAccount:
    def __init__(self, account_type, name, balance, rate_mu, rate_std=0, start_date='now'):
        self.account_type = account_type
        self._counter = 0
        # montly simulation
        self.month_delta = dateutil.relativedelta.relativedelta(months=1)
        
        # declare memory
        self.history = []
        self.name = name

        # read inputs
        # - convert yearly rate to monthly rate
        self.Rate_mu  = rate_mu
        self.rate_mu  = RateConverter.mu_year2month(self.Rate_mu)
        self.Rate_std = rate_std
        self.rate_std = RateConverter.std_year2month(self.Rate_std)
        self._rate = np.random.normal(self.rate_mu, self.rate_std, 1)[0]
        
        self.balance = balance
        if start_date == 'now':
            dt = datetime.datetime.now()
            self.start_date = startOfMonth(dt)
        # end if
        self.timepoint = self.start_date
        
        self.commit()
    # end def

    def run(self): # run for one month
        self._counter += 1
        
        self.timepoint += self.month_delta
        self._rate = np.random.normal(self.rate_mu, self.rate_std, 1)[0]
        
        self.balance = self.balance * self._rate
        self.commit()
    # end def
    
    def commit(self):
        self.history.append({
            'time': self.timepoint,
            'rate': self._rate,
            'counter': self._counter,
            'balance': self.balance
        }) # end history
    # end def
# end class

In [6]:
class LoanAccount(GeneralAccount):
    def __init__(self, name, balance, rate, default_payment, start_date='now'):
        super().__init__(account_type='Loan', name=name, balance=balance, rate_mu=rate, start_date=start_date)
        self.default_payment = default_payment
    # end def
    
    def get_instalment_payment(self):
        return min(self.default_payment, self.balance)
    # end def
    
    def pay_instalment(self, amount):
        self.balance -= amount
        self._counter += 1
        self.commit()
    # end def
# end class

In [47]:
class InvestmentAccount(GeneralAccount):
    def __init__(self, name, balance, rate_mu, rate_std, start_date='now'):
        super().__init__(account_type='Investment', name=name, balance=balance, rate_mu=rate_mu, rate_std=rate_std, start_date=start_date)
    # end def
    
    def deposit(self, amount):
        assert amount >= 0
        self.balance += amount
        self._counter += 1
        self.commit()
    # end def
    
    def withdraw(self, amount):
        if self.balance < amount:
            _timept = self.timepoint.isoformat().split('T')[0]
            raise ValueError('[%s @ %s] Withdrawal amount more than account balance' % (self.name, _timept,))
        # end if
        self.balance -= amount
        self._counter += 1
        self.commit()
    # end def
# end class

In [49]:
class SavingAccount(InvestmentAccount):
    def __init__(self, name, balance, rate, start_date='now'):
        super().__init__(name, balance, rate_mu=rate, rate_std=0, start_date=start_date)
        self.account_type = 'Saving'
    # end def
# end class

In [102]:
Current_account = SavingAccount(name='Equity', balance=176000, rate=1.000)

In [103]:
TFL_loan = LoanAccount(name='Tuition Fee Loan', balance=2500, rate=1.0475, default_payment=4200/12)

In [104]:
Dad_loan = LoanAccount(name="Dad's Loan", balance=100000, rate=1.023, default_payment=2300/12)

In [105]:
Salary = GeneralAccount(name='Salary', account_type='income', balance=72000/12, rate_mu=1.0395)

In [106]:
Expense = GeneralAccount(name='Expense', account_type='expense', balance=17400/12, rate_mu=1.0285)

In [107]:
Cash_reserve = GeneralAccount(name='Cash Reserve', account_type='reserve', balance=5000, rate_mu=1.0285)

In [108]:
Stock_market = InvestmentAccount(name='Stock', balance=0, rate_mu=1.115, rate_std=0)

In [109]:
Bond_market = InvestmentAccount(name='Bond', balance=0, rate_mu=1.025, rate_std=0)

In [110]:
class allocator_template:
    def __init__(self, name):
        self.name = name
    # end def
    
    def allocate(self, amount):
        raise NotImplementedError('allocate method is not implemented')
    # end def
# end class

In [111]:
allocator = allocator_template(name='aggressive_allocator')
allocator.allocate = lambda amount: {'stock': amount, 'bond': 0}

In [112]:
def epoch():
    # receive salary
    Salary.run()

    # deposit salary into current account
    Current_account.deposit(Salary.balance)

    # accumulate expense to pay
    Expense.run()
    # withdraw from current account to pay expense
    Current_account.withdraw(Expense.balance)

    # loan 01 grows with interest rate
    TFL_loan.run()
    payment = TFL_loan.get_instalment_payment()
    # with money from current account to pay loan
    Current_account.withdraw(payment)
    TFL_loan.pay_instalment(payment)

    # loan 02 grows with interest rate
    Dad_loan.run()
    payment = Dad_loan.get_instalment_payment()
    # with money from current account to pay loan
    Current_account.withdraw(payment)
    Dad_loan.pay_instalment(payment)

    # estimate cash reserved needed
    Cash_reserve.run()
    reserve = Cash_reserve.balance

    # get the amount of investable fund
    investable_amount = Current_account.balance - reserve
    # withdraw from current account
    Current_account.withdraw(investable_amount)

    # allocate to stock and bond investment accounts
    investable_allocations = allocator.allocate(investable_amount)
    # deposit to respective account
    Stock_market.deposit(investable_allocations['stock'])
    Bond_market .deposit(investable_allocations['bond'])
    # investment product grows with presumed rate
    Stock_market.run()
    Bond_market.run()
# end def